In [1]:
#%pip install chatterbot-corpus -qq
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
import NER_Attempt_02 as ner


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/dan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/dan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
bot = ChatBot(
    'TravelJohnny',
    #storage_adapter='chatterbot.storage.SQLStorageAdapter',
    #database_uri='sqlite:///database.sqlite3'
)

conversation = [
  'Hello',
  'Hi there, where would you like to travel?',
  'I want to go to Italy',
  'For which dates would you like to go?',
  'July 19th to August 12th',
  'And what is your budget for this trip?',
  '2000 dollars',
  'Great, Here are some options:',
]

Sample_Text_1 = [['Hi there'],
               ['How can I help you?'],
               ['Are there any flights from Houston to San Diego'],
               ['Yes, for which dates would you like?'],
               ['June 3rd to June 18th'],
               ['How much would like to spend?'],
               ['Under 500'],
               ['Great, Here are some options:']]

conversation_list = []

for t in conversation:
    conversation_list.append([t])

travel_chat_preproc = ner.word_preproc(Sample_Text_1)
travel_ner_out = ner.word_ner_all(travel_chat_preproc)

print(travel_ner_out)



[nltk_data] Downloading package stopwords to /home/dan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/dan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


{'Locations': ['hi', 'houston', 'san diego'], 'Dates': [datetime.datetime(2020, 6, 3, 0, 0), datetime.datetime(2020, 6, 18, 0, 0)], 'Money': ['500']}


In [56]:
#Bring in George's conversations...





In [62]:
trainer = ListTrainer(bot)
trainer.train(conversation)

print('Hello, how can I help you?')

Locations_list = []
Dates_list = []
Money_list = []

cnt = 0

while True:
    try:
        
        #Parse the customer input
        cust_input = input()        
        travel_chat_preproc = ner.word_preproc([[cust_input]])
        travel_ner_out = ner.word_ner_all(travel_chat_preproc)   
        
        #See if we found any information
        if len(travel_ner_out['Locations'])>0:
            for a in travel_ner_out['Locations']:
                Locations_list.append(a) 
        
        if len(travel_ner_out['Dates'])>0:
            for b in travel_ner_out['Dates']:
                Dates_list.append(b) 
            
        if len(travel_ner_out['Money'])>0:
            for c in travel_ner_out['Money']:
                Money_list.append(c) 
        
        print(len(Locations_list),len(Dates_list),len(Money_list))
        
        if cnt<3:
            bot_input = bot.get_response(cust_input)
            print(bot_input)
        
        else:
            
            if len(Locations_list)<2:
                print('Where do you want to travel from?')
                
            elif len(Dates_list)<2:
                print('When do you want to return?')
                
            elif len(Money_list)<1:
                print('How much do you want to spend?')
        
            else:
                print('Great, here are some options:')
                #Use information above to search sky scanner
                break
        
        cnt = cnt + 1
 
    except(KeyboardInterrupt, EOFError, SystemExit):
        break

List Trainer: [####################] 100%
Hello, how can I help you?
I'd like to go to Las Vegas
1 0 0
July 19th to August 12thAnd what is your budget for this trip?
900 Dollars
1 0 1
Great, Here are some options:
Would you like to know which dates?
1 0 1
I want to go to ItalyFor which dates would you like to go?
July 20th
1 1 1
Where do you want to travel from?
Seattle
2 1 1
When do you want to return?
In 3 weeks
2 2 1
Great, here are some options:


In [57]:
# If no second loaction, ask for start location

# If no second date, assume one way

# If no money, assume cheapest

# If no dates, ask for dates, if not look at today/oneway

# If more than 3 locations -> Multicity

# Chatterbot
# https://chatterbot.readthedocs.io/en/stable/training.html


#Sky Scanner



# Integration with Slack

In [5]:
bot.export_for_training('my_export.json')

AttributeError: 'ChatBot' object has no attribute 'export_for_training'